In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_products = pd.read_excel('products.xlsx')
df_stock = pd.read_excel('stock_levels.xlsx')

In [4]:
df_products

,Product ID,Product Name,Description,Price
0,P001,phone,Smartphone,299.99
1,P002,Tablet,Android Tablet,149.99
2,P003,LAPTOP,NaN,NaN
3,P004,Monitor,HD Monitor,199.50
4,P005,Mouse,Wireless Mouse,49.99
5,P006,Keyboard,Mechanical Keyboard,89.99
6,P002,Tablet,Android Tablet,149.99
7,NaN,Charger,NaN,twenty


In [5]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0    Product ID   7 non-null      object
 1   Product Name  8 non-null      object
 2   Description   6 non-null      object
 3   Price         7 non-null      object
dtypes: object(4)
memory usage: 384.0+ bytes


In [6]:
df_products.columns = df_products.columns.str.strip().str.lower().str.replace(" ", "_")

In [7]:
df_products.columns

Index(['product_id', 'product_name', 'description', 'price'], dtype='object')

In [8]:
df_products = df_products.drop_duplicates(subset='product_name').reset_index(drop=True)

In [9]:
df_products['product_name'] = df_products['product_name'].str.strip().str.title()
df_products['description'] = df_products['description'].str.strip().str.title()

In [10]:
df_products['price'] = pd.to_numeric(df_products['price'], errors='coerce')

In [11]:
df_products['price'] = df_products['price'].fillna(df_products['price'].mean())

In [12]:
# replacing description nan values with mode
df_products['description'] = df_products['description'].fillna('No description available')

In [13]:
df_products = df_products.dropna(subset=['product_id'])

In [14]:
df_products

,product_id,product_name,description,price
0,P001,Phone,Smartphone,299.990
1,P002,Tablet,Android Tablet,149.990
2,P003,Laptop,No description available,157.892
3,P004,Monitor,Hd Monitor,199.500
4,P005,Mouse,Wireless Mouse,49.990
5,P006,Keyboard,Mechanical Keyboard,89.990


In [15]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    6 non-null      object 
 1   product_name  6 non-null      object 
 2   description   6 non-null      object 
 3   price         6 non-null      float64
dtypes: float64(1), object(3)
memory usage: 240.0+ bytes


In [16]:
df_stock.head(10)

,product_id,warehouse,stock_quantity
0,P001,Riyadh,10
1,P002,Jeddah,20
2,P001,Jeddah,fifteen
3,P003,Dammam,30
4,P004,Riyadh,25
5,P005,Dammam,10
6,P005,Riyadh,NaN
7,P006,Jeddah,5
8,P007,Riyadh,0


In [17]:
df_stock.columns = df_stock.columns.str.strip().str.lower().str.replace(' ','_')

In [18]:
df_stock = df_stock.drop(6).reset_index(drop=True)

In [19]:
df_stock['stock_quantity'] = df_stock['stock_quantity'].replace("fifteen", 15) # to comvert fifteen to 15
df_stock['stock_quantity'] = pd.to_numeric(df_stock['stock_quantity']) # to convert the column to numeric type

In [20]:
df_stock.head(10)

,product_id,warehouse,stock_quantity
0,P001,Riyadh,10
1,P002,Jeddah,20
2,P001,Jeddah,15
3,P003,Dammam,30
4,P004,Riyadh,25
5,P005,Dammam,10
6,P006,Jeddah,5
7,P007,Riyadh,0


In [21]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   product_id      8 non-null      object
 1   warehouse       8 non-null      object
 2   stock_quantity  8 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 320.0+ bytes


In [ ]:
stock_summary = df_stock.groupby('product_id', as_index=False)['stock_quantity'].sum() # sum stock quantity by product id
stock_summary.rename(columns={'stock_quantity': 'total_stock'}, inplace=True) # rename column to total_stock

In [ ]:
final_df = pd.merge(df_products, stock_summary, on='product_id', how='left') # left join 

In [ ]:
final_df = final_df[['product_id', 'product_name', 'price', 'total_stock']] # keeping only the columns we need

In [ ]:
final_df['stock_value'] = final_df['price'] * final_df['total_stock'] # Calculate the total value of the stock

In [ ]:
final_df.to_csv('cleaned_inventory.csv', index=False) # save to csv file

In [28]:
final_df.head(10)

,product_id,product_name,price,total_stock,stock_value
0,P001,Phone,299.990,25,7499.75
1,P002,Tablet,149.990,20,2999.80
2,P003,Laptop,157.892,30,4736.76
3,P004,Monitor,199.500,25,4987.50
4,P005,Mouse,49.990,10,499.90
5,P006,Keyboard,89.990,5,449.95
